# **Chap 7：自然语言处理**

In [1]:
import tensorflow as tf
import matplotlib
from matplotlib import pyplot as plt
import numpy as np
import collections
from source.code import ch7
from source.code import utils 

2023-06-02 21:20:43.332164: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-02 21:20:43.533461: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-02 21:20:43.565974: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-06-02 21:20:44.345916: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

In [2]:
# 如果你要在服务器上和别人共用GPU，可以设置你需要的显存资源
utils.gpu_limitation_config(memory=30,device=1)

## **7.2 词嵌入进阶**

### **7.2.1 全局向量词嵌入 GloVe**

一个文本序列窗口中的**词共现**（即两个词元共同出现）携带了丰富的语义信息：
* 例如“固体”比“气体”更有可能与“冰块”共现，而当提到“气体”时，“蒸汽”就会比“冰块”容易出现

在一个大型语料库中，**考虑整个全局语料库的词共现能获得更优秀的词嵌入效果**，在 `word2vec` 模型（以跳元模型为例）中，我们**仅从每个中心词左右扩展的上下文窗口出发，来对词元出现的概率建模**

下面，我们用 $q_{i,j}$ **表示模型预测的条件概率** $P(w_j|w_i)$：
$$
q_{i,j} = \frac{ \exp(u_j^Tv_i) }{\sum_{k\in\mathcal{V}} \exp(u_k^Tv_i) }
$$
考虑词元 $w_i$ 在整个语料库中可能出现多次，所有以 $w_i$ 为中心词的上下文词形成一个词索引多重集 $C_i$
* 该索引允许同一元素的多个实例，对于任意元素 $x\in C_i$，其**实例数称为重数**
* 例如中心词元 $c$ 全局出现过两次，上下文窗口分别是 $\{k,j,c,m,k\},\{k,l,c,k,j\}$，则多重集 $C = \{j,j,k,k,k,k,l,m\}$，元素 $k$ 的重数为 4
* 用 $x_{i,j}$ 表示多重集 $C_i$ 中元素 $j$ 的重数，则 $x_{i,j}$ 表示**全局语料中，上下文词 $w_j$ 和中心词 $w_i$ 的共现次数**，使用**全局语料统计的跳元模型损失函数**（**负对数似然**）为：
    $$
    -\sum_{i\in\mathcal{V}}\sum_{j\in\mathcal{V}} x_{i,j} \log q_{i,j}
    $$
* 如果我们用 $x_i$ 表示以 $w_i$ 为中心词的所有上下文词的个数，即 $x_i = |C_i|$，用 $p_{i,j} = x_{i,j} / x_i$ 表示**用于生成上下文词 $w_j$ 的条件概率**，则损失函数可以改写为：
    $$
    -\sum_{i\in\mathcal{V}}x_i \sum_{j\in\mathcal{V}} p_{i,j}\log q_{i,j}
    $$
  * 内部求和项 $-\sum_j p_{i,j}\log q_{i,j}$ 是根据**语料统计的条件分布** $p_{i,j}$ 和**模型建模预测的条件分布** $q_{i,j}$ 的**交叉熵损失**
  * 外层求和，通过 $x_i$ 对内层损失做加权求和
  * 最小化上述损失，可以理解为**让模型预测的条件分布接近全局语料库统计的条件分布**

注意，**交叉熵损失函数**通常用于建模两个分布之间的距离，但上述建模中使用交叉熵损失可能不是一个好的选择，因为：
* 规范化概率 $q_{i,j}$ 需要进行 `softmax` 操作，这在上一节中讨论过，**计算代价非常昂贵**
* 在大型语料库中，大量罕见事件往往会被交叉熵损失建模，这会**导致厚尾事件赋予了过多的权重**

基于以上思考，全局向量词嵌入模型 `GloVe` 基于**平方损失** 和 `word2vec` **跳元模型** 做出了三个修改：
* 使用变量 $p_{i,j}^{\prime} = x_{i,j}$ 和 $q_{i,j}^{\prime} = \exp(u_j^Tv_i)$ **而非使用概率分布**，并取两者的对数，所以**平方损失**是：
  $$
  \left(p_{i,j}^{\prime}-  q_{i,j}^{\prime} \right)^2 = \left( u_j^Tv_i - \log x_{i,j} \right)^2
  $$
* 为每个词元的**词向量表示添加了偏置项**
* 用某个权重函数 $h(x_{i,j})$ 替换每个损失项的权重，$h(x)$ 在区间 $[0,1]$ 内递增

综上，GloVe 模型的损失函数为极小化：
$$
\sum_{i\in\mathcal{V}} \sum_{j\in\mathcal{V}}  h(x_{i,j}) \left( u_j^Tv_i + b_i + c_j - \log x_{i,j} \right)^2
$$
其中，$b_i,c_j$ 分别是中心词 $w_i$ 和上下文词 $w_j$ 的偏置项

权重函数 $h(x)$ 的设计是，选取某个阈值 $c > 0$，然后：
$$
h(x)=\left\{
    \begin{array}{ll}
    (x/c)^a & ,x < c\\
    1 & ,x \geq c\\
    \end{array}
\right.
$$
其中，幂次 $a$ 通常取 $a=0.75$，由于 $h(0) = 0$
* 为了提高计算效率，可以**省略任意** $x_{i,j} = 0$ **的平方损失项**（即忽略没有任何共现的项）
* 参数的学习依然使用批量随机梯度下降
* 注意 $x_{i,j}$ 在全局语料库中进行统计，因此 `GloVe` 得到的**词嵌入向量称为全局向量**
* 注意 $w_i$ 出现在词 $w_j$ 的上下文窗口时，$w_j$ 也出现在 $w_i$ 的上下文窗口，因此 $x_{i,j} = x_{j,i}$ ，这意味着 `GloVe` 拟合**对称条件概率** $\log x_{i,j}$，这与 `word2vec` 的非对称条件概率 $p_{i,j}$ 是不同的
  * 这意味着在数学上，`GloVe` 的**中心词向量和上下文词向量表示是等价的**
  * 但由于随机初始化的原因，最终训练收敛时，词元 $w_i$ 的这两个表示向量 $u_i,v_i$ 通常结果不同
  * `GloVe` 选择将 $u_i,v_i$ **相加得到最终的词嵌入表示**

我们来简单解释上面的**平方损失是如何引入**的
* 考虑共现概率 $p_{i,j} = P(w_j|w_i), p_{i,k} = P(w_k|w_i)$，以及共现概率的比值 $p_{i,j} / p_{i,k}$，有以下四种情况：
  * 当 $w_i$ 与 $w_j$ 相关，与 $w_k$ 不相关，预计 $p_{i,j} / p_{i,k} > 1$ 会比较大
  * 当 $w_i$ 与 $w_k$ 相关，与 $w_j$ 不相关，预计 $p_{i,j} / p_{i,k} < 1$ 会接近 0
  * 当 $w_i$ 同时与 $w_k,w_j$ 相关，预计 $p_{i,j} / p_{i,k} $ 会接近 1
  * 当 $w_i$ 同时与 $w_k,w_j$ 不相关，预计 $p_{i,j} / p_{i,k} $ 会接近 1
* 因此，共现概率比值 $p_{i,j} / p_{i,k}$ 也**蕴含了词与词之间的关系**，我们考虑直接用中心词向量 $v_i$，上下文词向量 $u_j,u_k$ 来**建模共现概率比值**，拟合某个函数 $f$ 使得：
  $$
  f(v_i,u_j,u_k)\approx \frac{p_{i,j}}{p_{i,k}}
  $$
* $f$ 的设计需要满足几个简单的要求，首先 $f$ 是**标量函数**，例如 $f(v_i,u_j,u_k) = f((u_j - u_k)^Tv_i)$，其次，交换索引 $j,k$，需要保持 $f(x)f(-x) = \frac{p_{i,j}}{p_{i,k}}\times \frac{p_{i,k}}{p_{i,j}} = 1$，所以一种选择是 $f(x) = \exp(x)$，得到：
  $$
  f(v_i,u_j,u_k) = \frac{\exp (u_j^Tv_i)}{\exp(u_k^Tv_i)} \approx \frac{p_{i,j}}{p_{i,k}}
  $$
* 现在进一步假设 $\exp(u_j^Tv_i) \approx ap_{i,j}$，其中 $a$ 是常数，回忆 $p_{i,j} = x_{i,j} / x_i$，两边取对数得到：
  $$
  u_j^Tv_i \approx \log a + \log x_{i,j} - \log x_i
  $$
  我们用**额外的偏置项拟合** $-\log a + \log x_i$，就得到了：
  $$
  u_j^Tv_i + b_i + c_j \approx u_j^Tv_i + log x_i - \log a \approx \log x_{i,j}
  $$
* 因此，`GloVe` 的损失考虑 $\left(u_j^Tv_i + b_i + c_j - \log x_{i,j}\right)^2$

### **7.2.2 子词嵌入**

在文本中，无论是中文还是英文，都存在很多词，它们具有相同的“**词根**“
* 例如 `help`，`helps`，`helped`
* 例如 `他们的`，`她们的`，`它们的`，表达的语义相似

但在 `word2vec` 和 `GloVe` 模型中，这些相似的词将会使用完全不同的词向量来表达
* 对于**相似但出现频次较少的词元**，模型学习得到的词向量表示效果可能不好


**(1) fastText 模型**

为了使用这种相似词的**形态信息**，`fastText` 模型提出了**子词嵌入**方法，子词是一个**字符的 n-gram**，`fastText` 可以认为是**子词级别的跳元模型**，简单来说
* 我们放弃学习单词的词向量表示，而是**学习子词的词向量表示**
* 最终每个**单词的词向量由组成它的子词的词向量之和表示**

以单词 `great` 为例，在考虑它的中心词表示时，首先在单词开头和结尾添加 `<,>` 以将前缀、后缀与其他子词区分开，然后从中提取 n-gram 字符
* 例如 $n=3$ 时，提取子词有：`"<gr", "gre", "rea", "eat", "at>"`，另外包含一个特殊子词 `"<great>"`

现在，对于任意词元 $w$，设 $\mathcal{G}_w$ 表示长度为 $n=3$ 到 $n=6$ 之间的所有子词与特殊子词构成的集合，**词表是所有子词的集合**，假设 $z_g$ 表示词表中子词 $g$ 的词向量，则跳元模型中，词元 $w$ 的中心词向量 $v_w$ **表示为子词向量的和**：
$$
v_w = \sum_{g\in\mathcal{G}_w} z_g
$$

`fastText` 模型的其他部分与 `word2vec` 跳元模型相同，需要注意：
* 与 `word2vec` 相比，`fastText` **词表规模更大**，模型参数也更多，因此为了得到一个词的表示，带来了**更高的计算复杂度**
* 但现在，**相似结构的词之间共享来自它们子词的参数**，罕见词和词表外的词在 `fastText` 中也能得到很好的表示了

**(2) 字节对编码 Byte Pair Encoding**

在 `fastText` 中，所有子词都必须是固定长度，例如 $n=3$ 到 $n=6$，这在应用时多少会带来些不便，我们下面介绍一种**可变长度的子词编码**方案，称为**字节对编码**（**Byte Pair Encoding**，**BPE**），它能够帮助我们提取子词

BPE 对训练数据集中的词元进行统计，**发现单词内的公共符号**
* 从长度为 1 的符号开始，BPE **迭代地合并最频繁出现的连续字符**，以**产生新的更长的子词符号**
* 为了提高效率，BPE 可以**不考虑跨越单词边界的字符对**
* 最后，我们用子词来切分单词，得到单词的子词表示，切分规则是**贪心地让词元用更长的子词来表示**

BPE 及其变体目前是最强大的主流语言模型（例如 GPT）对文本的输入表示，现在，**模型很少直接使用词元本身来学习它们的嵌入**

注意，下面的演示<font color=red>**仅展示 BPE 的执行逻辑，真正实际场景中使用 BPE，还需小心处理文本中可能出现的各种特殊字符**</font>

以英文为例，首先，我们**初始化子词符号表** `symbols`
* 它包含长度为 1 的所有单个字母，以及**特殊词尾符号** `</w>`，和特殊的未知词元符号 `<unk>`
* 词尾符号 `</w>` 可以**帮助我们从编码后的输出序列恢复到原始的句子**，例如：
    ```python
    # 子词编码后的句子
    output = ["the</w>", "high", "est</w>", "mount", "ain</w>"]
    # 只需要用空格替换 </w> 就能恢复原句子
    print("".join(output).replace("</w>", " "))
    
    the highest mountain 
    ```

In [17]:
# 初始化子词符号表
symbols = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n',
           'o','p','q','r','s','t','u','v','w','x','y','z','</w>','<unk>']

由于不考虑跨越边界的符号对，我们考虑语料库中单词的词频，用 `raw_token_freqs` 存储
* 注意，统计词频时，每个单词尾要添加特殊符号 `</w>`

In [19]:
raw_token_freqs = {"fast</w>": 4, "faster</w>": 3, "tall</w>": 5, "taller</w>": 4}

由于子词提取是从单个字符开始的，所以初始化子词频率 `token_freqs` 时，需要**在每个字符之间插入空格作为分隔符**

In [20]:
token_freqs = {}
for token, freq in raw_token_freqs.items():
    # 用空格将单词分割成子词，然后加上结束符
    token_freqs[" ".join(list(token[:-4])) + " </w>"] = freq
token_freqs

{'f a s t </w>': 4,
 'f a s t e r </w>': 3,
 't a l l </w>': 5,
 't a l l e r </w>': 4}

下面的函数 `get_max_freq_pair()` 是 **BPE 的核心函数**，我们将**迭代的调用它**，返回**词内最频繁出现的连续符号对**
* 这些词来自于 `token_freqs` 的键
* 然后我们可以将连续符号对添加到子词符号表 `symbols`

In [21]:
def get_max_freq_pair(token_freqs):
    # 初始化字符对统计表
    pairs = collections.defaultdict(int)
    for token, freq in token_freqs.items():
        symbols = token.split() # 拆分token，得到子词符号
        # 对连续的符号对进行计数
        for i in range(len(symbols)-1):
            pairs[symbols[i],symbols[i+1]] += freq
    return max(pairs, key=pairs.get) # 返回最大频率的字符对

函数 `merge_symbols()` 用来**合并最频繁出现的连续符号对**，以产生新的符号

In [22]:
def merge_symbols(max_freq_pair, token_freqs, symbols):
    # 在子词符号表中添加新的字符对
    symbols.append("".join(max_freq_pair))
    # 初始化新的 token 频率表
    new_token_freqs = {}
    for token, freq in token_freqs.items():
        # 将 token 中的字符对替换为连续字符对
        # 例如 max_freq_pair = ('t', 'a')
        # token = 't a l l e r </w>' -> 'ta l l e r </w>'
        new_token = token.replace(" ".join(max_freq_pair), "".join(max_freq_pair))
        new_token_freqs[new_token] = freq
    return new_token_freqs

下面，我们迭代调用 10 词字节对编码 BPE，每次迭代我们**打印最频繁出现的连续符号对**，并更新 `token_freqs`

In [23]:
num_merges = 10
for i in range(num_merges):
    max_freq_pair = get_max_freq_pair(token_freqs)
    token_freqs = merge_symbols(max_freq_pair, token_freqs, symbols)
    print("#{}轮合并: {}".format(i+1,max_freq_pair))

#1轮合并: ('t', 'a')
#2轮合并: ('ta', 'l')
#3轮合并: ('tal', 'l')
#4轮合并: ('f', 'a')
#5轮合并: ('fa', 's')
#6轮合并: ('fas', 't')
#7轮合并: ('e', 'r')
#8轮合并: ('er', '</w>')
#9轮合并: ('tall', '</w>')
#10轮合并: ('fast', '</w>')


可以观察此时的子词符号表，额外添加了 10 个子词表示，**随着合并次数的增加，子词的长度会越来越长**

In [24]:
print(symbols)

['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '</w>', '<unk>', 'ta', 'tal', 'tall', 'fa', 'fas', 'fast', 'er', 'er</w>', 'tall</w>', 'fast</w>']


以及在经过 10 次 BPE 的子词抽取合并后，词频 `token_freqs` 中的单词，现在**也都由长度更长的子词来表示了**

In [25]:
print(token_freqs)

{'fast</w>': 4, 'fast er</w>': 3, 'tall</w>': 5, 'tall er</w>': 4}


最后，函数 `segment_BPE()` 用贪心方法，尝试将给定单词 `tokens` 使用子词符号表 `symbols` 中尽可能长的子词来表示
* 这意味着，我们希望一个单词**用尽可能少的子词来编码它**
* 在子词符号表 `symbols` 在数据集 $A$ 上学习得到后，我们完全可以用它来编码另一个数据集 $B$ 的单词

In [31]:
def segment_BPE(tokens : list, symbols : list):
    """
    ## segment_BPE
        BPE子词编码算法
    
    Parameters
    ----------
    tokens : list
        待子词编码的 token 列表
    symbols : list
        BPE 子词符号表
    """
    outputs = [] # 初始化输出
    for token in tokens:
        # 滑动窗口法，每次贪心地选择尽可能长的子词
        start, end = 0, len(token)
        cur_output = []
        while start < len(token) and start < end:
            if token[start:end] in symbols:
                cur_output.append(token[start:end])
                start = end
                end = len(token)
            else:
                # 不断缩小窗口右侧边界
                # 直到 token[start:end] 在子词符号表中
                end -= 1 
        # 如果 start < len(token)，说明上述循环由 start < end 跳出
        # 说明当前 token 无法被 BPE 子词符号表编码，出现了未知符号
        if start < len(token):
            cur_output.append('<unk>')
        outputs.append(' '.join(cur_output))
    return outputs

在之前的数据集中，我们没有 `tallest` 和 `fatter`，但我们仍然可以用 BPE 来编码它们

In [32]:
tokens = ['tallest</w>', 'fatter</w>']
print(segment_BPE(tokens, symbols))

['tall e s t </w>', 'fa t t er</w>']


我们仅介绍了一种最简单的 BPE 方案，在更复杂的实现中，**BPE 还要考虑跨越边界的连续字符**

### **7.2.3 词的相似性和词类比**

在 **7.1节**，我们在一个小型语料库数据集中训练了 `word2vec` 跳元模型，并尝试用它来寻找与给定词含义相近的词
* 在**大型语料库上预训练过的词向量表示，可以很好地应用到下游的各种 NLP 任务**

我们在这里展示两种简单的 NLP 任务，词的**相似性**和**词类比**任务

我们从 `GloVe` **下载和导入预训练过的** `GloVe` 嵌入表示，或者下载 `fastText` 的嵌入表示
* `GloVe` 模型有多种预训练结果，根据**所使用的语料库规模**和**词嵌入维度**分类
* 例如 `glove.6B.50d` 表示在词汇数量为 6B 的数据集上训练，词嵌入维度设定为 50

In [3]:
path = "../source/data/text/GloVe/glove.6B.50d/vec.txt"
file = open(path, 'r', encoding='utf-8')
print(file.readline())

the 0.418 0.24968 -0.41242 0.1217 0.34527 -0.044457 -0.49688 -0.17862 -0.00066023 -0.6566 0.27843 -0.14767 -0.55677 0.14658 -0.0095095 0.011658 0.10204 -0.12792 -0.8443 -0.12181 -0.016801 -0.33279 -0.1552 -0.23131 -0.19181 -1.8823 -0.76746 0.099051 -0.42125 -0.19526 4.0071 -0.18594 -0.52287 -0.31681 0.00059213 0.0074449 0.17778 -0.15897 0.012041 -0.054223 -0.29871 -0.15749 -0.34758 -0.045637 -0.44251 0.18785 0.0027849 -0.18411 -0.11514 -0.78581



我们从本地加载词嵌入文件，然后将词嵌入向量包装为**分词器** `TokenizerEmbedding`，它提供一些与词表 `Vocab` 相似的功能

In [4]:
class TokenizerEmbedding:
    def __init__(self, path : str):
        # path : 词嵌入文件路径
        self.idx_to_token, self.idx_to_vec = self._load_embedding(path)
        self.unk_idx = 0 # 未知词元的索引
        # 交换词元索引和词元的位置，得到 token_to_idx
        self.token_to_idx = {token:idx for idx, token in enumerate(self.idx_to_token)}

    def _load_embedding(self, path : str):
        # 从 path 加载词嵌入结果，并依次创建 idx_to_token 和 idx_to_vec
        idx_to_token, idx_to_vec = ['<unk>'], []
        with open(path, 'r', encoding='utf-8') as f:
            # 每一行是一个词元及其对应的词向量
            # 例如：'hello 0.32 0.12 ...'
            for line in f:
                line = line.rstrip()
                if line:
                    idx_to_token.append(line.split()[0]) # 词元
                    idx_to_vec.append(np.array(line.split()[1:], dtype=np.float32)) # 词向量
        # 在 idx_to_vec 中添加一个全 0 向量表示未知词元的嵌入
        idx_to_vec.insert(0, [0.0]*len(idx_to_vec[0]))
        idx_to_vec = np.stack(idx_to_vec) # 拼接得到词向量矩阵
        return idx_to_token, idx_to_vec
    
    # 定义下标访问方法
    def __getitem__(self, tokens : list):
        if isinstance(tokens, str):
            idx = self.token_to_idx.get(tokens, self.unk_idx)
            return self.idx_to_vec[idx]
        vectors = [self.__getitem__(token) for token in tokens]
        return np.array(vectors)
    
    def __len__(self):
        return len(self.idx_to_token)

In [5]:
# 创建词元编码器
glove_6b50d = TokenizerEmbedding(path)
print("词元数量：", len(glove_6b50d))

词元数量： 400001


我们可以通过 `token_to_idx` 获得某个词元在词表中的索引，然后用 `idx_to_token` 进行逆变换

In [6]:
token = "hello"
idx = glove_6b50d.token_to_idx[token]
print("{} 的索引是：{}".format(token, idx))
print("{} 的词向量是：\n{}".format(token, glove_6b50d[token]))
print("索引 {} 的词元是：{}".format(idx, glove_6b50d.idx_to_token[idx]))

hello 的索引是：13076
hello 的词向量是：
[-0.38497001  0.80092001  0.064106   -0.28354999 -0.026759   -0.34531999
 -0.64253002 -0.11729    -0.33256999  0.55242997 -0.087813    0.90350002
  0.47102001  0.56656998  0.69849998 -0.35229    -0.86541998  0.90573001
  0.03576    -0.071705   -0.12327     0.54922998  0.47005001  0.35572001
  1.26110005 -0.67580998 -0.94983     0.68665999  0.38710001 -1.34920001
  0.63511997  0.46416    -0.48813999  0.83827001 -0.92460001 -0.33722001
  0.53741002 -1.06159997 -0.081403   -0.67110997  0.30923    -0.39230001
 -0.55001998 -0.68826997  0.58048999 -0.11626     0.013139   -0.57653999
  0.048833    0.67203999]
索引 13076 的词元是：hello


**(1) 词相似性**

接下来，我们可以利用 `sklearn` 中的**最邻近算法**，例如 `KNN`，自定义距离的度量，**从而快速在词表中找到与给定词含义最相似的其他词**
* 距离的度量可以定义为：**1 - 余弦相似度**
* 在上一节的实现中，**每给定一个词元，我们需要计算它和所有其他词元的距离，然后对相似度进行排序**，这在频繁的查找中**非常低效**，通过最邻近算法，我们就可以在类似 `KD 树` 一样的数据结构中快速查询相似的单词了

In [10]:
from sklearn.neighbors import NearestNeighbors

In [11]:
class CosineSimilarity:
    def __init__(self, embed, topK : int=10) -> None:
        self.embed = embed # 词嵌入模型
        self.topK = topK # 返回最相似的 topK 个词元
        self.vocab_size = len(embed) # 词元数量

        # 余弦相似度距离
        def cosine_distance(x, y):
            # x, y : 词元的索引
            # embed : 词嵌入模型
            # 计算 cos 相似度
            x, y = int(x), int(y) # 转换为整数
            # 获得词向量
            x_vec = self.embed.idx_to_vec[x]
            y_vec = self.embed.idx_to_vec[y]
            # 距离为 1 - cos 相似度
            return 1 - np.dot(x_vec, y_vec) / (np.linalg.norm(x_vec) * np.linalg.norm(y_vec) + 1e-16)

        # 训练
        self.model = NearestNeighbors(n_neighbors=topK,metric=cosine_distance)
        self.model.fit(np.arange(self.vocab_size).reshape(-1,1))
    
    def kneighbors(self, token : str):
        # token : 词元
        idx = self.embed.token_to_idx.get(token, 0) # 词元索引
        # 返回最相似的 topK 个词元的索引
        distance, indices = self.model.kneighbors([[idx]])
        distance, indices = distance[0], indices[0] # 去除掉多余的样本维度
        
        similarities = 1 - distance # 将距离转换为相似度
        tokens = [self.embed.idx_to_token[idx] for idx in indices] # 将索引转换为词元
        
        # 打印查找结果
        for s,token in zip(similarities, tokens):
            print("词元：{}，相似度：{:.3f}".format(token, s))
        return tokens

接下来，**实例化模型，进行训练**

In [12]:
knn = CosineSimilarity(glove_6b50d,topK=10)

给定单词，就可以进行高效查询了，每次无需再计算相似度，再排序

In [14]:
tokens = knn.kneighbors("chip")

词元：chip，相似度：1.000
词元：chips，相似度：0.856
词元：intel，相似度：0.749
词元：electronics，相似度：0.749
词元：semiconductor，相似度：0.731
词元：maker，相似度：0.716
词元：computer，相似度：0.708
词元：microprocessor，相似度：0.707
词元：makers，相似度：0.703
词元：micro，相似度：0.701


In [15]:
tokens = knn.kneighbors("beautiful")

词元：beautiful，相似度：1.000
词元：lovely，相似度：0.921
词元：gorgeous，相似度：0.893
词元：wonderful，相似度：0.830
词元：charming，相似度：0.825
词元：beauty，相似度：0.801
词元：elegant，相似度：0.774
词元：looks，相似度：0.758
词元：love，相似度：0.736
词元：graceful，相似度：0.735


**(2) 词类比**

词类比是经典 NLP 中的一种有趣任务，对于给定的两组单词对 `a:b :: c:d`，但给出前三个词 `a, b, c` 时，我们需要在词表中找到 `d`，例如：
* 给定 `man : woman`，以及 `c = son`，类比可得 `d = daughter`


具体算法上，当给定 `a, b, c` 后，用 `vec(w)` 表示词 `w` 的词向量，则我们在词表中寻找 `d` 使得 `vec(d)` 与 `vec(b) - vec(a) + vec(c)` 最相似，函数 `get_analogy()` 用于**实现词类比任务**

In [38]:
def get_analogy(token_a, token_b, token_c, embed):
    def find_nearest(weights, target):
        # weights : 所有词元的词向量
        # 计算余弦相似度
        cos = (weights @ target[:, None])[:,0] / tf.sqrt(tf.reduce_sum(weights * weights, axis=1) 
                                                         * tf.reduce_sum(target * target) + 1e-9)
        # 通过 argsort 函数排序
        top_token = tf.argsort(cos, axis=0, direction='DESCENDING')[0].numpy()
        return top_token

    # 转换为词向量
    vec_a, vec_b, vec_c = embed[token_a], embed[token_b], embed[token_c]
    target = vec_b - vec_a + vec_c # 计算目标向量
    return embed.idx_to_token[find_nearest(embed.idx_to_vec, target)]

让我们来测试几个例子，此类比**除了能够包含语义、还可以类比时态、比较级**

In [42]:
questions = [["man","woman","son"],
             ["beijing","china","tokyo"],
             ["bad","worst","big"],
             ["do","did","go"],
             ["he","his","she"]]
for q in questions:
    a, b, c = q
    d = get_analogy(a,b,c,glove_6b50d)
    print("{} 和 {} 类比 {} 和 {}".format(a,b,c,d))

man 和 woman 类比 son 和 daughter
beijing 和 china 类比 tokyo 和 japan
bad 和 worst 类比 big 和 biggest
do 和 did 类比 go 和 went
he 和 his 类比 she 和 her


----

### **练习**

----